# Fine-tuning IndicBERT for Sanskrit Loanword Detection

This notebook demonstrates how to fine-tune the IndicBERT model for detecting Sanskrit loanwords in various Indic languages.

In [ ]:
!pip install transformers datasets torch numpy pandas tqdm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
import pandas as pd
from tqdm.auto import tqdm

## Load IndicBERT Model and Tokenizer

In [ ]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

## Prepare Dataset

Load your prepared dataset containing words labeled as Sanskrit loanwords or not.

In [ ]:
# Load and preprocess the dataset
dataset = load_dataset('json', data_files={'train': 'train.json', 'validation': 'validation.json'})

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

## Define Training Arguments and Metrics

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

## Train the Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics
)

trainer.train()

## Save the Fine-tuned Model

In [ ]:
# Save model and tokenizer
model_save_path = './fine_tuned_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

## Evaluate Model Performance

In [ ]:
# Run evaluation
eval_results = trainer.evaluate()
print("Evaluation Results:")
print(json.dumps(eval_results, indent=2))